# Trade accounting

This notebook decomposes gross exports into value added categories according to the Borin and Mancini (2019) trade accounting framework.

| Category | Formula | Description |
| ---- | ------- | ----------- |
| `DAVAX1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{y}_{sr}$ | Domestic value added (DVA) completed in $s$ and sent to $r$ |
| `DAVAX2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \mathbf{y}_{rr}$ | DVA in intermediates sent to, completed by, and absorbed in $r$ |
| `REX1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{y}_{ru}$ | DVA in intermediates sent to and completed by $r$ then exported to third economy |
| `REX2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{A}_{ru} \sum_k \sum_{\ell \neq s,r} \mathbf{B}_{uk} \mathbf{y}_{k\ell}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in third economy |
| `REX3` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{A}_{ru} \sum_k \mathbf{B}_{uk} \mathbf{y}_{kr}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in $r$ |
| `REF1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \mathbf{y}_{rs}$ | DVA in intermediates sent to and completed by $r$ then exported to $s$ |
| `REF2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r} \mathbf{A}_{ru} \sum_k \mathbf{B}_{uk} \mathbf{y}_{ks}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in $s$ |
| `FVA` | $\sum_{t \neq s} \mathbf{v}_t \mathbf{B}^{(s)}_{ts} \mathbf{e}_{sr}$ | Foreign value added (FVA) in gross exports |
| `PDC1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \sum_{s \neq u} \mathbf{A}_{su} \mathbf{B}_{us} \mathbf{e}_{sr}$ | Pure double counting (PDC) of domestic origin |
| `PDC2` | $\sum_{t \neq s} \mathbf{v}_t \mathbf{B}^{(s)}_{ts} \sum_{s \neq u} \mathbf{A}_{su} \mathbf{B}_{us} \mathbf{e}_{sr}$ | PDC of foreign origin |

Sectors are broken down either by exporting sectors (es) or by the origin of value added (os). As such, only the export sector breakdown is technically an exports decomposition in that the decomposed terms sum to gross exports. Both sector breakdowns aggregate to the same country-level values.

Before running this notebook, select the MRIO version to load from `data/mrio/` as well as the corresponding output filename to be saved in `data/`.

## Setup

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

### Select MRIO version

In [2]:
input, output = 'adb-mrio.parquet', 'ta.parquet'
# input, output = 'adb-mrio62.parquet', 'ta62.parquet'
# input, output = 'adb-mrio62-const.parquet', 'ta62-const.parquet'

### Parameters

In [3]:
sectors = pd.read_excel('../dicts/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

In [4]:
ta_es, ta_os = pd.DataFrame(), pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd, Yf = zeroout(Y, inverse=True), zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    A = Z @ np.diag(np.where(x != 0, 1/x, 0))
    Ad, Af = zeroout(A, inverse=True), zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        # Breakdown by export sectors

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s) @ subset(Bnots, s, s))
        VB_FC = np.diag(subset(v, -s) @ subset(Bnots, -s, s))
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = np.diag(subset(v, s) @ subset(Bnots, s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)
        PDC2 = np.diag(subset(v, -s) @ subset(Bnots, -s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)

        ta_es_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'es',
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'exports': asvector(Exports),
            'davax1': asvector(DAVAX1), 'davax2': asvector(DAVAX2),
            'rex1': asvector(REX1), 'rex2': asvector(REX2), 'rex3': asvector(REX3),
            'ref1': asvector(REF1), 'ref2': asvector(REF2),
            'fva': asvector(FVA), 'pdc1': asvector(PDC1), 'pdc2': asvector(PDC2)
        })
        ta_es = pd.concat([ta_es, ta_es_s], ignore_index=True)

        # Breakdown by origin sectors

        VB_DC = np.diag(subset(v, s)) @ subset(Bnots, s, s)
        VB_FC = diagrow(subset(v, -s)) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_os_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'os',
            'i': np.tile(sectors['ind'], G-1), 
            'i5': np.tile(sectors['ind5'], G-1), 
            'i15': np.tile(sectors['ind15'], G-1),
            'exports': np.nan,
            'davax1': asvector(DAVAX1), 'davax2': asvector(DAVAX2),
            'rex1': asvector(REX1), 'rex2': asvector(REX2), 'rex3': asvector(REX3),
            'ref1': asvector(REF1), 'ref2': asvector(REF2),
            'fva': asvector(FVA), 'pdc1': asvector(PDC1), 'pdc2': asvector(PDC2)
        })
        ta_os = pd.concat([ta_os, ta_os_s], ignore_index=True)

    print(f'{year} done')

ta = pd.concat([ta_es, ta_os], ignore_index=True)
ta.to_parquet(f'../data/{output}', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


### View results

In [5]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()

,t,s,r,breakdown,i,i5,i15,exports,davax1,davax2,rex1,rex2,rex3,ref1,ref2,fva,pdc1,pdc2
0,2017,1,2,es,1,1,1,1.060398,0.859037,0.033666,0.021076,0.022030,0.000227,0.000124,0.000217,0.121546,0.002141,0.000335
1,2017,1,2,es,2,1,2,19.591016,0.096988,8.667168,1.867850,6.175448,0.070163,0.007332,0.047063,2.613973,0.038895,0.006135
2,2017,1,2,es,3,2,3,2.940326,1.481226,0.355159,0.345620,0.374473,0.005009,0.001428,0.003875,0.368191,0.004617,0.000727
3,2017,1,2,es,4,2,3,25.800999,0.623895,5.427990,6.090037,8.673722,0.100024,0.025762,0.109010,4.698245,0.045173,0.007139
4,2017,1,2,es,5,2,3,5.168567,0.141941,0.943591,1.177794,1.576127,0.019096,0.004934,0.018924,1.272189,0.011985,0.001986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207515,2022,73,72,os,31,5,13,NaN,6.147921,4.032670,1.342418,0.754698,0.001526,0.167253,0.083681,7.326459,0.177037,0.079674
2207516,2022,73,72,os,32,5,14,NaN,6.720624,2.475536,0.493052,0.311558,0.000638,0.060902,0.036048,2.905143,0.061875,0.030648
2207517,2022,73,72,os,33,5,14,NaN,2.933578,2.104256,0.540417,0.307589,0.000610,0.065676,0.032942,1.302012,0.053850,0.013589
2207518,2022,73,72,os,34,5,15,NaN,12.162483,9.557576,3.220068,1.882014,0.003781,0.406433,0.213710,9.447234,0.248481,0.096777
